In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from cmapPy.pandasGEXpress.parse import parse
from scipy.stats import spearmanr as scor
from scipy.stats import mannwhitneyu as mwu

**Downloading LINCS data** \
From Gene Expression Omnibus downloading the [GSE92742](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE92742) and the [GSE70138](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE70138) datasets. From both datasets I dowloaded the Level5 gene expression profiles, the metadata from the sig_info files and a gene info file.\
I downloaded the drug indformation and the sample information for  compound metadata from the [Drug Repurposing Hub](https://clue.io/repurposing)

In [5]:
#checking if the files are in the correct directory
import os
files_needed_GSE92742=['GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
               'GSE92742_Broad_LINCS_sig_info.txt','GSE92742_Broad_LINCS_gene_info.txt']
files_needed_GSE70138=['GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328.gctx',
                      'GSE70138_Broad_LINCS_sig_info.txt']
files_needed_rep=['repurposing_drugs_20180907.txt','repurposing_samples_20180907.txt']
files_GSE92742=os.listdir('../data/GSE92742/')
files_GSE70138=os.listdir('../data/GSE70138/')
files_rep=os.listdir('../data/repurposing/')
for f in files_needed_GSE92742:
    assert (f in files_GSE92742)
for f in files_needed_GSE70138:
    assert (f in files_GSE70138)
for f in files_needed_rep:
    assert (f in files_rep)

**Importing gene ids** 

In [8]:
gene_ids=pd.read_csv('../data/GSE92742/GSE92742_Broad_LINCS_gene_info.txt',
                    sep='\t',header=0,index_col=0)
gene_ids.head()

,pr_gene_symbol,pr_gene_title,pr_is_lm,pr_is_bing
pr_gene_id,,,,
780,DDR1,discoidin domain receptor tyrosine kinase 1,1,1
7849,PAX8,paired box 8,1,1
2978,GUCA1A,guanylate cyclase activator 1A,0,0
2049,EPHB3,EPH receptor B3,0,1
2101,ESRRA,estrogen related receptor alpha,0,1


In [9]:
fil=gene_ids['pr_is_lm']==1
gene_ids=gene_ids[fil]
gene_ids=gene_ids['pr_gene_symbol']
gene_ids.head()

pr_gene_id
780      DDR1
7849     PAX8
6193     RPS5
23      ABCF1
9552    SPAG7
Name: pr_gene_symbol, dtype: object

In [10]:
gene_ids.index=gene_ids.index.astype(str)

In [11]:
#To calculate consensus signature,I used the MODZ method
#described in the original LINCS manuscript.
def calc_MODZ(data):
    if len(data)==1:
        return data.iloc[0]
    if len(data)==2:
        return np.mean(data,0)
    else:
        CM=scor(data.T)[0]
        fil=CM<0
        CM[fil]=0.01
        weights=np.sum(CM,1)-1
        weights=weights/np.sum(weights)
        weights=weights.reshape((-1,1))
        return pd.Series(np.dot(data.T,weights).reshape((-1,1)[0]),index=data.columns)

**Importing ligand-receptor network**\
I downloaded the receptor-ligand network from [here](https://zenodo.org/record/3260758/files/lr_network.rds), and preprocessed it in R. From it I choose the ligand and receptor genes

In [13]:
#column 'to': receptors, column 'from': ligands
ligand_receptor=pd.read_csv('../data/lr_network.csv', sep=',', header=0, index_col=0)

In [14]:
good_sources=['kegg_cytokines', 'kegg_neuroactive','pharmacology', 'ramilowski_known']

In [15]:
fil=np.in1d(ligand_receptor['source'], good_sources)
ligand_receptor=ligand_receptor[fil]

In [16]:
receptors=ligand_receptor['to'].unique()
ligands=ligand_receptor['from'].unique()

In [17]:
l_r=list(receptors)+list(ligands)

**Getting the drugs from the Drug Repurposing Hub**

In [18]:
drugs=pd.read_csv('../data/repurposing/repurposing_drugs_20180907.txt', sep='\t', header=0, index_col=None, encoding='latin', skiprows=9)

In [20]:
fil=drugs['target'].isnull()
drugs=drugs[~fil]

In [21]:
drugs.head()

,pert_iname,clinical_phase,moa,target,disease_area,indication
0,"[sar9,met(o2)11]-substance-p",Preclinical,tachykinin antagonist,TACR1,NaN,NaN
1,A-1070722,Preclinical,glycogen synthase kinase inhibitor,GSK3A|GSK3B,NaN,NaN
2,A-1120,Preclinical,retinoid receptor ligand,RBP4,NaN,NaN
3,A-317491,Preclinical,purinergic receptor antagonist,P2RX3,NaN,NaN
5,A-366,Preclinical,histone lysine methyltransferase inhibitor,EHMT1|EHMT2,NaN,NaN


In [22]:
#creating a file that has the drug, its target and the target type
def split_dataframe(one_line):
    if '|' in one_line['target']:
        targets=one_line['target'].split('|')
        temp=pd.DataFrame(index=range(len(targets)), columns=one_line.index)
        for col in temp.columns:
            temp[col]=one_line[col]
        temp['target']=targets
        return temp
    else:
        return pd.DataFrame(one_line).T

In [35]:
results=pd.DataFrame(columns=drugs.columns)
for i in drugs.index:
    one_line=drugs.loc[i]
    results=pd.concat([results, split_dataframe(one_line)])

In [36]:
results.to_csv('../results/lincs_drugs.csv', sep=',')

In [37]:
results.head()

,pert_iname,clinical_phase,moa,target,disease_area,indication
0,"[sar9,met(o2)11]-substance-p",Preclinical,tachykinin antagonist,TACR1,NaN,NaN
0,A-1070722,Preclinical,glycogen synthase kinase inhibitor,GSK3A,NaN,NaN
1,A-1070722,Preclinical,glycogen synthase kinase inhibitor,GSK3B,NaN,NaN
2,A-1120,Preclinical,retinoid receptor ligand,RBP4,NaN,NaN
3,A-317491,Preclinical,purinergic receptor antagonist,P2RX3,NaN,NaN


In [38]:
results.shape

(13097, 6)

In [39]:
d=list(results['pert_iname'].unique())
l_r_d=l_r + d

In [40]:
filt0=np.in1d(results['target'],l_r_d)
results=results[filt0]

In [41]:
results.shape

(2518, 6)

In [42]:
activators=['agonist', 'activator', 'stimulant', 'enhancer', 'reactivator', 'inducer']
inhibitors=['inhibitor', 'antagonist', 'blocker', 'downregulator', 'destabilizer']

In [43]:
fil=~results['moa'].isna()
results=results[fil]

In [44]:
#arrange them in order
results.index=range(len(results.index)) 

In [47]:
#Giving a sign for each drug-target pairs (antagonists: -1, agonists: +1)
results['activator']=0
results['inhibitor']=0
for i in results.index:
    moa=results.loc[i, 'moa']
    is_a=len(set(moa.split()) &set (activators))
    is_i=len(set(moa.split()) &set (inhibitors))
    results.loc[i,['activator', 'inhibitor']]=is_a, is_i

In [48]:
results.head()

,pert_iname,clinical_phase,moa,target,disease_area,indication,activator,inhibitor
0,"[sar9,met(o2)11]-substance-p",Preclinical,tachykinin antagonist,TACR1,NaN,NaN,0,1
1,A-987306,Preclinical,histamine receptor antagonist,AVPR1A,NaN,NaN,0,1
2,A-987306,Preclinical,histamine receptor antagonist,CCR1,NaN,NaN,0,1
3,A-987306,Preclinical,histamine receptor antagonist,HTR1A,NaN,NaN,0,1
4,A-987306,Preclinical,histamine receptor antagonist,HTR1B,NaN,NaN,0,1


In [49]:
results.to_csv('../results/lincs_drugs_act_inhib.csv', sep=',')

**Importing the gse92742 and gse70138 files**

In [52]:
gse92742=pd.read_csv('../data/gse92742/GSE92742_Broad_LINCS_sig_info.txt', sep='\t', header=0, index_col=0) 
gse70138=pd.read_csv('../data/gse70138/GSE70138_Broad_LINCS_sig_info.txt', sep='\t', header=0, index_col=0) 

In [51]:
gse92742['pert_type'].unique() 

array(['ctl_vehicle', 'trt_cp', 'ctl_untrt', 'trt_sh.cgs',
       'ctl_vehicle.cns', 'ctl_vector.cns', 'ctl_untrt.cns', 'trt_sh.css',
       'trt_lig', 'ctl_vector', 'trt_sh', 'trt_oe', 'trt_oe.mut'],
      dtype=object)

In [54]:
gse70138['pert_type'].unique()

array(['ctl_vehicle', 'trt_cp', 'trt_xpr', 'ctl_untrt', 'ctl_vector'],
      dtype=object)

In [53]:
#filering by perturbation type
good_92742=['trt_sh.cgs','trt_lig','trt_cp','trt_oe']
fil1=np.in1d(gse92742['pert_type'], good_92742)
gse92742=gse92742[fil1]

good_70138=['trt_cp','trt_xpr']
fil2=np.in1d(gse70138['pert_type'], good_70138)
gse70138=gse70138[fil2]

In [56]:
#filtering receptors and ligans
filt1=np.in1d(gse92742['pert_iname'], l_r_d) 
gse92742=gse92742[filt1]

filt2=np.in1d(gse70138['pert_iname'], l_r_d)
gse70138=gse70138[filt2]

In [62]:
#Giving signs for the perturbations:
#CRIPSR, shRNA : -1 
#ligand, overexpression: +1
gse70138['sign']=0
gse92742['sign']=0

fil_xpr=gse70138['pert_type']=='trt_xpr'
gse70138.loc[gse70138.index[fil_xpr],'sign']=-1
fil_sh=gse92742['pert_type']=='trt_sh.cgs',
gse92742.loc[gse92742.index[fil_sh],'sign']=-1
fil_lig=gse92742['pert_type']=='trt_lig'
gse92742.loc[gse92742.index[fil_lig],'sign']=+1
fil_oe=gse92742['pert_type']=='trt_oe'
gse92742.loc[gse92742.index[fil_oe],'sign']=+1

In [63]:
gse70138.to_csv('../results/LINCS_gse70138.csv', sep=',')
gse92742.to_csv('../results/LINCS_gse92742.csv', sep=',')

**Creating a seperate file for each perturbations**

In [64]:
fil_xpr2=gse70138['pert_type']=='trt_xpr'
gse70138_trt_xpr=gse70138[fil_xpr2]
gse70138_trt_xpr.to_csv('../results/LINCS_gse70138_trt_xpr.csv', sep=',')

In [65]:
fil_cp1=gse70138['pert_type']=='trt_cp'
gse70138_trt_cp=gse70138[fil_cp1]
gse70138_trt_cp.to_csv('../results/LINCS_gse70138_trt_cp.csv', sep=',')

In [66]:
fil_cp2=gse92742['pert_type']=='trt_cp'
gse92742_trt_cp=gse92742[fil_cp2]
gse92742_trt_cp.to_csv('../results/LINCS_gse92742_trt_cp.csv', sep=',')

In [67]:
fil_sh2=gse92742['pert_type']=='trt_sh.cgs'
gse92742_trt_sh=gse92742[fil_sh2]
gse92742_trt_sh.to_csv('../results/LINCS_gse92742_trt_sh.csv', sep=',')

In [68]:
fil_lig2=gse92742['pert_type']=='trt_lig'
gse92742_trt_lig=gse92742[fil_lig2]
gse92742_trt_lig.to_csv('../results/LINCS_gse92742_trt_lig.csv', sep=',')

In [69]:
fil_oe2=gse92742['pert_type']=='trt_oe'
gse92742_trt_oe=gse92742[fil_oe2]
gse92742_trt_oe.to_csv('../results/LINCS_gse92742_trt_oe.csv', sep=',')

**Creating consensus singatures**

In [ ]:
gse70138_trt_xpr=pd.read_csv('../results/LINCS_gse70138_trt_xpr.csv', sep=',',header=0, index_col=0, low_memory=False)

In [ ]:
#rows are the perturbed genes, columns are the measured landmark genes from LINCS
genes_perturbed=gse70138_trt_xpr['pert_iname'].unique()
consensus_signatures_gse70138_trt_xpr=pd.DataFrame(index=genes_perturbed,columns=gene_ids.index.astype(str))
consensus_signatures_gse70138_trt_xpr.head()

In [ ]:
for i in range(len(genes_perturbed)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed[i]
    fil=gse70138_trt_xpr['pert_iname']==gene
    samples=gse70138_trt_xpr.index[fil]
    expression=parse('../data/gse70138/GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328.gctx',
                 cid=samples,rid=gene_ids.index.astype(str)).data_df.T[gene_ids.index.astype(str)]
    consensus_signatures_gse70138_trt_xpr.loc[gene]=calc_MODZ(expression)

In [ ]:
consensus_signatures_gse70138_trt_xpr.columns=gene_ids[consensus_signatures_gse70138_trt_xpr.columns].values

In [ ]:
consensus_signatures_gse70138_trt_xpr.to_csv('../results/consensus_signature_gse70138_trt_xpr.csv',sep=',')

In [ ]:
gse70138_trt_cp=pd.read_csv('../results/LINCS_gse70138_trt_cp.csv', sep=',',header=0, index_col=0, low_memory=False)

In [ ]:
genes_perturbed1=gse70138_trt_cp['pert_iname'].unique()
consensus_signatures_gse70138_trt_cp=pd.DataFrame(index=genes_perturbed1,columns=gene_ids.index)
consensus_signatures_gse70138_trt_cp.head()

In [ ]:
for i in range(len(genes_perturbed1)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed1[i]
    fil=gse70138_trt_cp['pert_iname']==gene
    samples=gse70138_trt_cp.index[fil]
    expression=parse('../data/gse70138/GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse70138_trt_cp.loc[gene]=calc_MODZ(expression)

In [ ]:
consensus_signatures_gse70138_trt_cp.columns=gene_ids[consensus_signatures_gse70138_trt_cp.columns].values

In [ ]:
consensus_signatures_gse70138_trt_cp.to_csv('../results/consensus_signature_gse70138_trt_cp.csv',sep=',')

In [ ]:
gse92742_trt_cp=pd.read_csv('../results/LINCS_gse92742_trt_cp.csv', sep=',',header=0, index_col=0, low_memory=False)

In [ ]:
genes_perturbed2=gse92742_trt_cp['pert_iname'].unique()
consensus_signatures_gse92742_trt_cp=pd.DataFrame(index=genes_perturbed2,columns=gene_ids.index)
consensus_signatures_gse92742_trt_cp.head()

In [ ]:
for i in range(len(genes_perturbed2)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed2[i]
    fil=gse92742_trt_cp['pert_iname']==gene
    samples=gse92742_trt_cp.index[fil]
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse92742_trt_cp.loc[gene]=calc_MODZ(expression)

In [ ]:
consensus_signatures_gse92742_trt_cp.head()

In [ ]:
consensus_signatures_gse92742_trt_cp.columns=gene_ids[consensus_signatures_gse92742_trt_cp.columns].values

In [ ]:
consensus_signatures_gse92742_trt_cp.to_csv('../results/consensus_signature_gse92742_trt_cp.csv',sep=',')

In [ ]:
gse92742_trt_sh=pd.read_csv('../results/LINCS_gse92742_trt_sh.csv', sep=',',header=0, index_col=0, low_memory=False)

In [ ]:
genes_perturbed3=gse92742_trt_sh['pert_iname'].unique()
consensus_signatures_gse92742_trt_sh=pd.DataFrame(index=genes_perturbed3,columns=gene_ids.index)
consensus_signatures_gse92742_trt_sh.head()

In [ ]:
for i in range(len(genes_perturbed3)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed3[i]
    fil=gse92742_trt_sh['pert_iname']==gene
    samples=gse92742_trt_sh.index[fil]
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse92742_trt_sh.loc[gene]=calc_MODZ(expression)

In [ ]:
consensus_signatures_gse92742_trt_sh.head()

In [ ]:
consensus_signatures_gse92742_trt_sh.columns=gene_ids[consensus_signatures_gse92742_trt_sh.columns].values

In [ ]:
consensus_signatures_gse92742_trt_sh.to_csv('../results/consensus_signature_gse92742_trt_sh.csv',sep=',')

In [ ]:
gse92742_trt_sh_fil=pd.read_csv('../results/LINCS_gse92742_trt_sh_fil.csv', sep=',',header=0, index_col=0, low_memory=False)

In [ ]:
genes_perturbed7=gse92742_trt_sh_fil['pert_iname'].unique()
consensus_signatures_gse92742_trt_sh_fil=pd.DataFrame(index=genes_perturbed7,columns=gene_ids.index)
consensus_signatures_gse92742_trt_sh_fil.head()

In [ ]:
for i in range(len(genes_perturbed7)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed7[i]
    fil=gse92742_trt_sh_fil['pert_iname']==gene
    samples=gse92742_trt_sh_fil.index[fil]
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse92742_trt_sh_fil.loc[gene]=calc_MODZ(expression)

In [ ]:
consensus_signatures_gse92742_trt_sh_fil.head()

In [ ]:
consensus_signatures_gse92742_trt_sh_fil.columns=gene_ids[consensus_signatures_gse92742_trt_sh_fil.columns].values

In [ ]:
consensus_signatures_gse92742_trt_sh_fil.to_csv('../results/consensus_signature_gse92742_trt_sh_fil.csv',sep=',')

In [ ]:
gse92742_trt_lig=pd.read_csv('../results/LINCS_gse92742_trt_lig.csv', sep=',',header=0, index_col=0, low_memory=False)

In [ ]:
genes_perturbed4=gse92742_trt_lig['pert_iname'].unique()
consensus_signatures_gse92742_trt_lig=pd.DataFrame(index=genes_perturbed4,columns=gene_ids.index)
consensus_signatures_gse92742_trt_lig.head()

In [ ]:
for i in range(len(genes_perturbed4)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed4[i]
    fil=gse92742_trt_lig['pert_iname']==gene
    samples=gse92742_trt_lig.index[fil]
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse92742_trt_lig.loc[gene]=calc_MODZ(expression)

In [ ]:
consensus_signatures_gse92742_trt_lig.head()

In [ ]:
consensus_signatures_gse92742_trt_lig.columns=gene_ids[consensus_signatures_gse92742_trt_lig.columns].values

In [ ]:
consensus_signatures_gse92742_trt_lig.to_csv('../results/consensus_signature_gse92742_trt_lig.csv',sep=',')

In [ ]:
gse92742_trt_oe=pd.read_csv('../results/LINCS_gse92742_trt_oe.csv', sep=',',header=0, index_col=0, low_memory=False)

In [ ]:
genes_perturbed5=gse92742_trt_oe['pert_iname'].unique()
consensus_signatures_gse92742_trt_oe=pd.DataFrame(index=genes_perturbed5,columns=gene_ids.index)
consensus_signatures_gse92742_trt_oe.head()

In [ ]:
for i in range(len(genes_perturbed5)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed5[i]
    fil=gse92742_trt_oe['pert_iname']==gene
    samples=gse92742_trt_oe.index[fil]
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse92742_trt_oe.loc[gene]=calc_MODZ(expression)

In [ ]:
consensus_signatures_gse92742_trt_oe.head()

In [ ]:
consensus_signatures_gse92742_trt_oe.columns=gene_ids[consensus_signatures_gse92742_trt_oe.columns].values

In [ ]:
consensus_signatures_gse92742_trt_oe.to_csv('../results/consensus_signature_gse92742_trt_oe.csv',sep=',')